In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

#Establishing connection to the URL of TOP 250 Movies rated in IMBD

url = "https://www.imdb.com/chart/top?ref_=ft_250"
raw = get(url).content
html = bs(raw,"html.parser")
print(raw)


b'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    \n    \n    \n\n    \n    \n    \n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">\n            <style>\n                body#styleguide-v2 {\n                    background: no-repeat fixed center top #000;\n                }\n            </style>\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>IMDb Top 250 - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_tit

In [2]:
#Retrieving the Movies title
mov_lst=[]
list_items = html.find_all('td',{'class':'titleColumn'})
for i, el in enumerate(list_items):
     string = (el.text.replace('\n',''))
     string =re.sub('\d+\D\s','',string).split('(')
     mov_lst.append(string)
     #print(string)
#Create a DataFrame and rename Columns
df=pd.DataFrame(data=mov_lst)
df.rename(columns={0:'Title',1:'Year of Production'}, inplace = True)

#Clean year Column
df['Year of Production']=[el.replace(')','') for el in df['Year of Production']]

#Add Rating Scores
score_lst=[]
list_items = html.find_all('td',{'class':"ratingColumn imdbRating"})
for i, el in enumerate(list_items):
    score=(el.text.replace('\n',''))
    score_lst.append(score)
df2 = df.assign(Rating = score_lst)
#df2

#Add Actors to df2
cast_lst=[]
list_items = html.find_all('td',{'class':'titleColumn'})
for i, el in enumerate(list_items):
    cast=el.find('a')['title']
    cast_=cast.split(',')
    cast_lst.append(cast_)
#Remove Director from cast
#Add list of Cast to df2
for i in cast_lst:
    del i[0]
df3 = df2.assign(Cast = cast_lst)

#Split the Cast into to Columns
df3=(df3.Cast.apply(pd.Series).merge(df3, left_index = True, right_index = True))
df4 = df3.drop(['Cast'], axis=1)
df4.rename(columns={0:'Cast_1',1:'Cast_2'}, inplace = True)
df5=df4[['Title','Year of Production','Rating','Cast_1','Cast_2']]
df5.head(20)


,Title,Year of Production,Rating,Cast_1,Cast_2
0,The Shawshank Redemption,1994,9.2,Tim Robbins,Morgan Freeman
1,The Godfather,1972,9.1,Marlon Brando,Al Pacino
2,The Godfather: Part II,1974,9.0,Al Pacino,Robert De Niro
3,The Dark Knight,2008,9.0,Christian Bale,Heath Ledger
4,12 Angry Men,1957,8.9,Henry Fonda,Lee J. Cobb
5,Schindler's List,1993,8.9,Liam Neeson,Ralph Fiennes
6,The Lord of the Rings: The Return o...,2003,8.9,Elijah Wood,Viggo Mortensen
7,Pulp Fiction,1994,8.9,John Travolta,Uma Thurman
8,"The Good, the Bad and the Ugly",1966,8.8,Clint Eastwood,Eli Wallach
9,Joker,2019,8.8,Joaquin Phoenix,Robert De Niro


In [27]:
#Most Ranked Actors
actors_rank=df5['Cast_1'].value_counts().to_frame()
actors_rank.head(20)


,Cast_1
Robert De Niro,6
Tom Hanks,5
Aamir Khan,5
Charles Chaplin,5
Leonardo DiCaprio,5
Clint Eastwood,4
James Stewart,4
Christian Bale,4
Harrison Ford,3
Toshirô Mifune,3
